In [1]:
import pandas as pd
import numpy as np
import snowflake.connector

In [2]:
#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_PROD',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [28]:
#Get work logs
sql = f'''
select 
coalesce(metadata:pvid,concat(metadata:domain, '!', metadata:item_id)) as PVID,
metadata:category as CATEGORY,
--REPLACE(REPLACE(y.value,'_form',''),'form_','') as ATTRIBUTE,
metadata:title as Title,
metadata:link as URL,
_id as DELIVERY_TASK,
REPLACE(x.key,'form_','') as ATTRIBUTE,
REPLACE(y.value:field_id::text,'form_','') as SUBFIELD,
case when ATTRIBUTE=REPLACE(y.value:field_id::text,'form_','') THEN 'response' ELSE 'is_cbd' END as RESPONSE_TYPE,
y.value:response[0]::text as TASKER_RESPONSE, 
concat(PVID,ATTRIBUTE,SUBFIELD) as TASKER_RESPONSE_ID
from SCALE_PROD.PUBLIC.TASKS,
lateral flatten(input => response:annotations) as x,
lateral flatten(input => x.value:response[0]) as y
where project='645dd187d8315806686babc4' and PVID IS NOT NULL and CATEGORY = 'Cell Phone Cases'
'''
cs.execute(sql)
worklogdf = cs.fetch_pandas_all()


In [43]:
#create a dataframe with PVID, Category, attribute, response_type, tasker_response
tdf = worklogdf[['PVID','CATEGORY','ATTRIBUTE','RESPONSE_TYPE','TASKER_RESPONSE']]
print(tdf)

                              PVID            CATEGORY  \
0       www.walmart.com!1642863478  "Cell Phone Cases"   
1       www.walmart.com!1642863478  "Cell Phone Cases"   
2       www.walmart.com!1642863478  "Cell Phone Cases"   
3       www.walmart.com!1642863478  "Cell Phone Cases"   
4       www.walmart.com!1642863478  "Cell Phone Cases"   
...                            ...                 ...   
391759   www.walmart.com!701799259  "Cell Phone Cases"   
391760   www.walmart.com!701799259  "Cell Phone Cases"   
391761   www.walmart.com!701799259  "Cell Phone Cases"   
391762   www.walmart.com!701799259  "Cell Phone Cases"   
391763   www.walmart.com!701799259  "Cell Phone Cases"   

                     ATTRIBUTE RESPONSE_TYPE   TASKER_RESPONSE  
0       assembledProductHeight        is_cbd              true  
1       assembledProductLength        is_cbd              true  
2       assembledProductWeight        is_cbd              true  
3        assembledProductWidth        is_cb

In [44]:
#remove all rows in tdf where RESPONSE_TYPE = response
tdf = tdf[tdf['RESPONSE_TYPE'] != 'response']
# remove all rows in tdf where TASKER_RESPONSE is not true or TASKER_RESPONSE is not FALSE
tdf = tdf[(tdf['TASKER_RESPONSE'] == 'true') | (tdf['TASKER_RESPONSE'] == 'false')]
print(tdf)

                              PVID            CATEGORY  \
0       www.walmart.com!1642863478  "Cell Phone Cases"   
1       www.walmart.com!1642863478  "Cell Phone Cases"   
2       www.walmart.com!1642863478  "Cell Phone Cases"   
3       www.walmart.com!1642863478  "Cell Phone Cases"   
5       www.walmart.com!1642863478  "Cell Phone Cases"   
...                            ...                 ...   
391754   www.walmart.com!441183367  "Cell Phone Cases"   
391757   www.walmart.com!701799259  "Cell Phone Cases"   
391758   www.walmart.com!701799259  "Cell Phone Cases"   
391760   www.walmart.com!701799259  "Cell Phone Cases"   
391762   www.walmart.com!701799259  "Cell Phone Cases"   

                     ATTRIBUTE RESPONSE_TYPE TASKER_RESPONSE  
0       assembledProductHeight        is_cbd            true  
1       assembledProductLength        is_cbd            true  
2       assembledProductWeight        is_cbd            true  
3        assembledProductWidth        is_cbd       

In [45]:
adf = tdf.groupby(['CATEGORY','ATTRIBUTE','RESPONSE_TYPE']).agg({'PVID':'count'})
print(tdf)

                              PVID            CATEGORY  \
0       www.walmart.com!1642863478  "Cell Phone Cases"   
1       www.walmart.com!1642863478  "Cell Phone Cases"   
2       www.walmart.com!1642863478  "Cell Phone Cases"   
3       www.walmart.com!1642863478  "Cell Phone Cases"   
5       www.walmart.com!1642863478  "Cell Phone Cases"   
...                            ...                 ...   
391754   www.walmart.com!441183367  "Cell Phone Cases"   
391757   www.walmart.com!701799259  "Cell Phone Cases"   
391758   www.walmart.com!701799259  "Cell Phone Cases"   
391760   www.walmart.com!701799259  "Cell Phone Cases"   
391762   www.walmart.com!701799259  "Cell Phone Cases"   

                     ATTRIBUTE RESPONSE_TYPE TASKER_RESPONSE  
0       assembledProductHeight        is_cbd            true  
1       assembledProductLength        is_cbd            true  
2       assembledProductWeight        is_cbd            true  
3        assembledProductWidth        is_cbd       

In [42]:
#save adf to csv
tdf.to_csv('CategoryPTAnalysisCellPhoneCases.csv')

In [33]:
print(adf)


                                                                         PVID
CATEGORY           ATTRIBUTE                              RESPONSE_TYPE      
"Cell Phone Cases" accessoriesIncluded                    is_cbd         6905
                   assembledProductHeight                 is_cbd         7701
                   assembledProductLength                 is_cbd         8107
                   assembledProductWeight                 is_cbd         8089
                   assembledProductWidth                  is_cbd         7681
                   attachmentStyle                        is_cbd         7137
                   brand                                  is_cbd         6917
                   capacity                               is_cbd          230
                   case_style                             is_cbd         7137
                   cell_phone_case_features               is_cbd         7137
                   cell_phone_case_type                   is_cbd